## LUT convert Tool 

 * Converting an approximate multiplier function into a C++ header
 * **Important** : Only 8bit singed multipliers are supported at the moment. So the header must be a 256x256 C-array

## Example conversion from EvoApprox library

In [ ]:
! curl -s "https://ehw.fit.vutbr.cz/evoapproxlib/v1.1?folder=multiplers/8x8_signed/pareto_pwr_mae&file=mul8s_1L2H.c&pyx=bash" | bash

In [ ]:
#copy here the cript from evo approx (python pyx method)

import pyximport
pyximport.install()
import mul8s_1L2H

def u2s(v): # 16b unsigned to 16b signed
    if v & 32768:
        return v - 65536
    return v

In [ ]:
# set approximate mult function as the imported library function name
# you can use any function that can give a result with axx_mult.mult(a,b)
axx_mult = mul8s_1L2H

In [ ]:
# set number of bits of the approx mult - only 8bit signed mult is supported at the moment
# DO NOT CHANGE
nbits = 8

In [ ]:
#run this script to write the header c file (*.h) and then move it to adapt/cpu-kernels/axx_mults
#use the header name as the arument to adapt layers

with open('mul8s_1L2H.h', 'w') as filehandle: 
    bits = int(pow(2,nbits))
    lut_size_str = str(bits)

    filehandle.write('#include <stdint.h>\n\n')
    filehandle.write('const int' + str(2*nbits) + '_t lut [' + lut_size_str + '][' + lut_size_str +'] = {')       
    
    for i in range (0,bits//2):
        filehandle.write('{')
        for j in range (0,bits//2):
            x = u2s(axx_mult.mul(i,j))
            filehandle.write('%s' % x)
            filehandle.write(', ')  
        for j in range (bits//2,bits):
            x = u2s(axx_mult.mul(i,(bits-j)*-1))
            filehandle.write('%s' % x)
            if j!=bits-1:
                filehandle.write(', ') 
        filehandle.write('},')  
        filehandle.write('\n')
        
    for i in range (bits//2,bits):
        filehandle.write('{')
        for j in range (0,bits//2):
            x = u2s(axx_mult.mul((bits-i)*-1,j))        
            filehandle.write('%s' % x)
            filehandle.write(', ')  
        for j in range (bits//2,bits):
            x = u2s(axx_mult.mul((bits-i),(bits-j)))
            filehandle.write('%s' % x)
            if j!=bits-1:
                filehandle.write(', ')
        if(i!=bits-1):        
            filehandle.write('},')
            filehandle.write('\n')
    filehandle.write('}};\n')        